In [1]:
import tensorflow as tf
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, transform
from modelVit import *
from data import *

os.chdir('C:/Users/nurib/PythonCodes/Mammogram-segmentation-main_v10')
vid="v023_5" #version id should match the file number 

Model: "Improved_ViT_Segmentation_3Classes"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 480, 240, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 480, 240, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 vi_t_encoder (ViTEncoder)      (None, 450, 64)      178112      ['input_1[0][0]']                
                                                                 

In [2]:

# labelVisualize ve COLOR_DICT fonksiyon/degiskenlerini projenize eklediğinizden emin olun.
# Örneğin:
# from utils import labelVisualize, COLOR_DICT

def process_and_save_image(input_image_path, output_image_path, model,
                           target_size=(480, 240, 3), flag_multi_class=True,
                           as_gray=True, num_class=3):
    
    # Resmi oku ve normalize et
    img = io.imread(input_image_path, as_gray=as_gray)
    img = img / 255.0
    img = transform.resize(img, target_size)
    img = np.reshape(img, (1,) + img.shape)  # Modelin beklediği format: (1, height, width, channels)
    
    # Model ile tahmin yap
    result = model.predict(img, verbose=0)
    result_image = result[0]
    
    # Sonuç işleme: çok sınıflı segmentasyon varsa labelVisualize fonksiyonu ile işleyin,
    # aksi takdirde doğrudan ilk kanalı kullanın.
    if flag_multi_class:
        processed_img = labelVisualize(num_class, COLOR_DICT, result_image)
    else:
        processed_img = result_image[:, :, 0]
    
    processed_img = processed_img.astype(np.uint8)
    
    # Sonucu kaydet
    io.imsave(output_image_path, processed_img)


In [3]:
 model = build_improved_vit_segmentation()
 model.load_weights("files/unet_fgt_seg_" + vid + ".hdf5")

In [4]:
from tqdm import tqdm

def clean_blue_inside_green_adaptive(mask_rgb, green_ratio_thresh=0.85, blue_area_ratio=0.005):
    """
    Yeşil alanın içindeki mavi lekeleri, alanı yeşil alanın belli oranından küçükse temizler.

    green_ratio_thresh: Çevrenin en az yüzde kaçı yeşil olmalı.
    blue_area_ratio: Mavi bileşenin alanı, yeşil alanın yüzde kaçı kadar olabilir.
    """
    green = (0, 255, 0)
    blue = (0, 0, 255)

    green_mask = np.all(mask_rgb == green, axis=-1).astype(np.uint8)
    blue_mask = np.all(mask_rgb == blue, axis=-1).astype(np.uint8)
    total_green = np.count_nonzero(green_mask)

    if total_green == 0:
        return mask_rgb  # yeşil alan yoksa işlem yapma

    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(blue_mask, connectivity=8)

    for lbl in range(1, num_labels):
        component_mask = (labels == lbl).astype(np.uint8)
        area = stats[lbl, cv2.CC_STAT_AREA]

        max_allowed_area = total_green * blue_area_ratio
        if area > max_allowed_area:
            continue  # çok büyükse dokunma

        # Çevresini analiz et
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        dilated = cv2.dilate(component_mask, kernel, iterations=1)
        boundary = dilated.astype(bool) & (~component_mask.astype(bool))

        if np.count_nonzero(boundary) == 0:
            continue  # sınır yoksa atla

        green_ratio = np.sum(green_mask[boundary]) / len(green_mask[boundary])
        if green_ratio >= green_ratio_thresh:
            mask_rgb[labels == lbl] = green  # yeşile çevir

    return mask_rgb

    
def _find_boundary_pixels(component_mask):
    """
    component_mask: Tek renkli bir bileşene ait (True/False) numpy array.
    Bu fonksiyon, bileşenin sınır piksel koordinatlarını döndürür.
    """
    kernel = np.ones((3, 3), np.uint8)
    eroded = cv2.erode(component_mask.astype(np.uint8), kernel, iterations=1)
    boundary = component_mask & (eroded == 0)
    return np.where(boundary)

def _fill_component_with_neighbor_color(mask_rgb, labels, label_idx):
    """
    mask_rgb: (H, W, 3) boyutlu renkli maske.
    labels: (H, W) boyutlu, bileşen etiketleri.
    label_idx: Doldurulacak bileşenin label numarası.
    
    Bu fonksiyon, 'label_idx' bileşeninin sınırındaki komşu piksellerin
    çoğunlukta görülen rengini alarak bileşeni o renge boyar.
    """
    component_mask = (labels == label_idx)
    by, bx = _find_boundary_pixels(component_mask)

    if len(by) == 0:
        mask_rgb[component_mask] = [0, 0, 0]
        return

    height, width = mask_rgb.shape[:2]
    neighbor_colors = []
    for y, x in zip(by, bx):
        for ny in range(y-1, y+2):
            for nx in range(x-1, x+2):
                if (0 <= ny < height) and (0 <= nx < width):
                    if not component_mask[ny, nx]:
                        neighbor_colors.append(tuple(mask_rgb[ny, nx]))
    if not neighbor_colors:
        mask_rgb[component_mask] = [0, 0, 0]
        return

    color_counts = {}
    for c in neighbor_colors:
        color_counts[c] = color_counts.get(c, 0) + 1
    majority_color = max(color_counts, key=color_counts.get)
    mask_rgb[component_mask] = majority_color

def _keep_largest_component_and_fill_others(mask_rgb, color):
    """
    mask_rgb: (H, W, 3) boyutlu maske (RGB formatında).
    color: İşlem yapılacak renk (örneğin, (255,0,0) kırmızı).
    
    Bu fonksiyon, ilgili renge ait tüm bağlı bileşenleri bulur,
    en büyük bileşeni korur; diğer bileşenleri komşu çoğunluk renginde doldurur.
    """
    color_mask = np.all(mask_rgb == color, axis=-1).astype(np.uint8)
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(color_mask, connectivity=8)
    if num_labels <= 1:
        return mask_rgb

    max_area = 0
    largest_label = None
    for lbl in range(1, num_labels):
        area = stats[lbl, cv2.CC_STAT_AREA]
        if area > max_area:
            max_area = area
            largest_label = lbl

    for lbl in range(1, num_labels):
        if lbl == largest_label:
            continue
        _fill_component_with_neighbor_color(mask_rgb, labels, lbl)
    return mask_rgb

def process_mask(
    mask_path,
    out_path=None,
    kernel_size=5,
    apply_closing=True,
    process_colors=[(255, 0, 0), (0, 255, 0), (0, 0, 255)]
):
    """
    mask_path : Girdi maske dosyası (RGB formatında, OpenCV BGR olarak okuyacaktır).
    out_path  : Sonuç maske dosyasının kaydedileceği yol (opsiyonel).
    kernel_size: Morfolojik closing için kernel boyutu.
    apply_closing: True ise, her renk için binary maskede closing uygulanır.
    process_colors: İşlem yapılacak renkler listesi.
    
    Fonksiyon akışı:
      1) Girdi maske RGB formatında okunur.
      2) İstenirse her renk için morfolojik closing uygulanır.
      3) Her bir renk için; en büyük bağlı bileşen korunur, diğerleri komşu çoğunluk rengiyle doldurulur.
      4) Sonuç out_path verilmişse diske kaydedilir.
    """
    mask_bgr = cv2.imread(mask_path)
    if mask_bgr is None:
        raise FileNotFoundError(f"Girdi maske '{mask_path}' okunamadı.")
    mask_rgb = cv2.cvtColor(mask_bgr, cv2.COLOR_BGR2RGB)

    if apply_closing:
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
        for color in process_colors:
            color_bin = np.all(mask_rgb == color, axis=-1).astype(np.uint8)
            color_bin_closed = cv2.morphologyEx(color_bin, cv2.MORPH_CLOSE, kernel)
            mask_rgb[color_bin == 1] = [0, 0, 0]
            mask_rgb[color_bin_closed == 1] = color

    for color in process_colors:
        mask_rgb = _keep_largest_component_and_fill_others(mask_rgb, color)

    mask_rgb = clean_blue_inside_green_adaptive(mask_rgb)
    
    if out_path:
        out_bgr = cv2.cvtColor(mask_rgb, cv2.COLOR_RGB2BGR)
        cv2.imwrite(out_path, out_bgr)

    return mask_rgb

def mask_out_red_blue(original_image_path, processed_mask_path, out_path=None):
    """
    Orijinal görüntüdeki kırmızı ve mavi alanları maskeleyip (siyah yapıp) sonucu döndürür.
    
    original_image_path: Orijinal görüntünün dosya yolu.
    processed_mask_path: İşlenmiş maske görüntüsünün dosya yolu.
    out_path: (Opsiyonel) Sonucun kaydedileceği dosya yolu.
    """
    # Orijinal görüntüyü oku ve RGB'ye çevir
    original_bgr = cv2.imread(original_image_path)
    if original_bgr is None:
        raise FileNotFoundError(f"Orijinal görüntü '{original_image_path}' bulunamadı.")
    original_rgb = cv2.cvtColor(original_bgr, cv2.COLOR_BGR2RGB)
    
    # İşlenmiş maske görüntüsünü oku ve RGB'ye çevir
    mask_bgr = cv2.imread(processed_mask_path)
    if mask_bgr is None:
        raise FileNotFoundError(f"Maske görüntüsü '{processed_mask_path}' bulunamadı.")
    mask_rgb = cv2.cvtColor(mask_bgr, cv2.COLOR_BGR2RGB)
    
    # Kırmızı ve mavi renk aralıklarını tanımla
    lower_red = np.array([240, 0, 0])
    upper_red = np.array([256, 30, 30])
    lower_blue = np.array([0, 0, 240])
    upper_blue = np.array([30, 30, 256])
    
    # Maske görüntüsünde kırmızı ve mavi bölgeleri belirle
    mask_red = cv2.inRange(mask_rgb, lower_red, upper_red)
    mask_blue = cv2.inRange(mask_rgb, lower_blue, upper_blue)
    
    # İki maskeyi birleştir (kırmızı veya mavi olan pikseller 255 olacak)
    combined_mask = cv2.bitwise_or(mask_red, mask_blue)
    
    # Eğer maske boyutu orijinal görüntü ile uyuşmuyorsa yeniden boyutlandır
    if combined_mask.shape != original_rgb.shape[:2]:
        combined_mask = cv2.resize(combined_mask, (original_rgb.shape[1], original_rgb.shape[0]),
                                   interpolation=cv2.INTER_NEAREST)
    
    # Orijinal görüntüde, combined_mask'in 255 (kırmızı/mavi) olduğu yerleri siyah yap
    masked_result = original_rgb.copy()
    masked_result[combined_mask == 255] = [0, 0, 0]
    
    # Sonucu dosyaya kaydet
    if out_path is not None:
        result_bgr = cv2.cvtColor(masked_result, cv2.COLOR_RGB2BGR)
        cv2.imwrite(out_path, result_bgr)
    
    return masked_result

In [ ]:
########## MASS TRAIN ##########
import pandas as pd 

# CSV dosyasını oku
merged_df = pd.read_csv(r'N:\CBISDDSM\massTrain\mass_train_updated.csv')

# İlk satırdaki tam dosya yolunu alalım
full_path = merged_df.loc[0, "File Location_y"]
print("Full path:", full_path)

# Sadece dizin kısmını almak için:
base_dir = os.path.dirname(full_path)
print("Base directory:", base_dir)

In [ ]:
"""
# İlk 15 örnek için işlem yapmak isterseniz:
for idx in range(15):
    full_path = merged_df.loc[idx, "File Location_y"]
    base_dir = os.path.dirname(full_path)
    print(f"\nRow {idx} base_dir: {base_dir}")
    
    # Bu base_dir ile dosya yollarını oluşturun:
    original_image_path = os.path.join(base_dir, "1-1.png")
    mask_input_path      = os.path.join(base_dir, "1-2.png")
    mask_processed_path  = os.path.join(base_dir, "1-3.png")
    result_path          = os.path.join(base_dir, "1-4.png")
    
    print("\nProcessing single image")

    process_and_save_image(original_image_path,mask_input_path,model);
    # 1) Maske üzerinde temizleme işlemi
    try:
        final_mask = process_mask(
            mask_path=mask_input_path,
            out_path=mask_processed_path,
            kernel_size=5,
            apply_closing=True,
            process_colors=[(255, 0, 0), (0, 255, 0), (0, 0, 255)]
        )
    except Exception as e:
        print(f"Maske işleme hatası: {e}")
    
    # 2) Orijinal görüntü üzerinde maskeyi kullanarak kırmızı/mavi alanları siyaha boya
    try:
        mask_out_red_blue(original_image_path, mask_processed_path, out_path=result_path)
    except Exception as e:
        print(f"Mask out işlemi hatası: {e}")
    
    print("Processing completed and files saved.")
    
    # 3) Sonuçları ekranda gösterme
    try:
        original_img = cv2.cvtColor(cv2.imread(original_image_path), cv2.COLOR_BGR2RGB)
        processed_mask_img = cv2.cvtColor(cv2.imread(mask_processed_path), cv2.COLOR_BGR2RGB)
        result_img = cv2.cvtColor(cv2.imread(result_path), cv2.COLOR_BGR2RGB)
    
        fig, axs = plt.subplots(1, 3, figsize=(18, 6))
        axs[0].imshow(original_img)
        axs[0].set_title("Orijinal 1-1.png")
        axs[0].axis('off')
    
        axs[1].imshow(processed_mask_img)
        axs[1].set_title("Processed Mask 1-3.png")
        axs[1].axis('off')
    
        axs[2].imshow(result_img)
        axs[2].set_title("Masked Result 1-4.png")
        axs[2].axis('off')
    
        plt.tight_layout()
        plt.show()
    except Exception as e:
        print(f"Görüntü gösterme hatası: {e}")
"""
# Görsel gösterimi aç/kapat
show_images = False

# İlk 15 örnek için işlem
#for idx in tqdm(range(15), desc="İşlem ilerlemesi", unit="resim",leave=True):
for idx in tqdm(range(len(merged_df)), desc="İşlem ilerlemesi", unit="resim", leave=True):
    full_path = merged_df.loc[idx, "File Location_y"]
    base_dir = os.path.dirname(full_path)
    
    original_image_path = os.path.join(base_dir, "1-1.png")
    mask_input_path      = os.path.join(base_dir, "1-2.png")
    mask_processed_path  = os.path.join(base_dir, "1-3.png")
    result_path          = os.path.join(base_dir, "1-4.png")

    try:
        # Segmentasyon tahmini yapılacaksa
        process_and_save_image(original_image_path, mask_input_path, model)
    except Exception as e:
        print(f"[{idx}] Tahmin hatası: {e}")
        continue

    try:
        # Temizleme işlemi
        final_mask = process_mask(
            mask_path=mask_input_path,
            out_path=mask_processed_path,
            kernel_size=5,
            apply_closing=True,
            process_colors=[(255, 0, 0), (0, 255, 0), (0, 0, 255)]
        )
    except Exception as e:
        print(f"[{idx}] Maske temizleme hatası: {e}")
        continue

    try:
        mask_out_red_blue(original_image_path, mask_processed_path, out_path=result_path)
    except Exception as e:
        print(f"[{idx}] Maskeleme hatası: {e}")
        continue

    if show_images:
        try:
            original_img = cv2.cvtColor(cv2.imread(original_image_path), cv2.COLOR_BGR2RGB)
            processed_mask_img = cv2.cvtColor(cv2.imread(mask_processed_path), cv2.COLOR_BGR2RGB)
            result_img = cv2.cvtColor(cv2.imread(result_path), cv2.COLOR_BGR2RGB)

            fig, axs = plt.subplots(1, 3, figsize=(18, 6))
            axs[0].imshow(original_img)
            axs[0].set_title("Orijinal 1-1.png")
            axs[0].axis('off')

            axs[1].imshow(processed_mask_img)
            axs[1].set_title("Processed Mask 1-3.png")
            axs[1].axis('off')

            axs[2].imshow(result_img)
            axs[2].set_title("Masked Result 1-4.png")
            axs[2].axis('off')

            plt.tight_layout()
            plt.show()
        except Exception as e:
            print(f"[{idx}] Görüntü gösterme hatası: {e}")


In [ ]:
########## MASS TEST ##########
# CSV dosyasını oku
merged_df_massTest = pd.read_csv(r'N:\CBISDDSM\massTest\mass_test_updated.csv')

# İlk satırdaki tam dosya yolunu alalım
full_path = merged_df_massTest.loc[0, "File Location_y"]
print("Full path:", full_path)

# Sadece dizin kısmını almak için:
base_dir = os.path.dirname(full_path)
print("Base directory:", base_dir)

In [ ]:
# Görsel gösterimi aç/kapat
show_images = False

# İlk 15 örnek için işlem
#for idx in tqdm(range(15), desc="İşlem ilerlemesi", unit="resim",leave=True):
for idx in tqdm(range(len(merged_df_massTest)), desc="İşlem ilerlemesi", unit="resim", leave=True):
    full_path = merged_df_massTest.loc[idx, "File Location_y"]
    base_dir = os.path.dirname(full_path)
    
    original_image_path = os.path.join(base_dir, "1-1.png")
    mask_input_path      = os.path.join(base_dir, "1-2.png")
    mask_processed_path  = os.path.join(base_dir, "1-3.png")
    result_path          = os.path.join(base_dir, "1-4.png")

    try:
        # Segmentasyon tahmini yapılacaksa
        process_and_save_image(original_image_path, mask_input_path, model)
    except Exception as e:
        print(f"[{idx}] Tahmin hatası: {e}")
        continue

    try:
        # Temizleme işlemi
        final_mask = process_mask(
            mask_path=mask_input_path,
            out_path=mask_processed_path,
            kernel_size=5,
            apply_closing=True,
            process_colors=[(255, 0, 0), (0, 255, 0), (0, 0, 255)]
        )
    except Exception as e:
        print(f"[{idx}] Maske temizleme hatası: {e}")
        continue

    try:
        mask_out_red_blue(original_image_path, mask_processed_path, out_path=result_path)
    except Exception as e:
        print(f"[{idx}] Maskeleme hatası: {e}")
        continue

    if show_images:
        try:
            original_img = cv2.cvtColor(cv2.imread(original_image_path), cv2.COLOR_BGR2RGB)
            processed_mask_img = cv2.cvtColor(cv2.imread(mask_processed_path), cv2.COLOR_BGR2RGB)
            result_img = cv2.cvtColor(cv2.imread(result_path), cv2.COLOR_BGR2RGB)

            fig, axs = plt.subplots(1, 3, figsize=(18, 6))
            axs[0].imshow(original_img)
            axs[0].set_title("Orijinal 1-1.png")
            axs[0].axis('off')

            axs[1].imshow(processed_mask_img)
            axs[1].set_title("Processed Mask 1-3.png")
            axs[1].axis('off')

            axs[2].imshow(result_img)
            axs[2].set_title("Masked Result 1-4.png")
            axs[2].axis('off')

            plt.tight_layout()
            plt.show()
        except Exception as e:
            print(f"[{idx}] Görüntü gösterme hatası: {e}")

In [7]:
import pandas as pd 
########## CALC TEST ##########
# CSV dosyasını oku
merged_df_calcTest = pd.read_csv(r'N:\CBISDDSM\calcTest\calc_test_updated.csv')

# İlk satırdaki tam dosya yolunu alalım
full_path = merged_df_calcTest.loc[0, "File Location_y"]
print("Full path:", full_path)

# Sadece dizin kısmını almak için:
base_dir = os.path.dirname(full_path)
print("Base directory:", base_dir)

Full path: N:\CBISDDSM\calcTest\CBIS-DDSM\Calc-Test_P_00038_LEFT_CC\08-29-2017-DDSM-NA-96009\1.000000-full mammogram images-63992\1-1.dcm
Base directory: N:\CBISDDSM\calcTest\CBIS-DDSM\Calc-Test_P_00038_LEFT_CC\08-29-2017-DDSM-NA-96009\1.000000-full mammogram images-63992


In [8]:
# Görsel gösterimi aç/kapat
show_images = False

# İlk 15 örnek için işlem
#for idx in tqdm(range(15), desc="İşlem ilerlemesi", unit="resim",leave=True):
for idx in tqdm(range(len(merged_df_calcTest)), desc="İşlem ilerlemesi", unit="resim", leave=True):
    full_path = merged_df_calcTest.loc[idx, "File Location_y"]
    base_dir = os.path.dirname(full_path)
    
    original_image_path = os.path.join(base_dir, "1-1.png")
    mask_input_path      = os.path.join(base_dir, "1-2.png")
    mask_processed_path  = os.path.join(base_dir, "1-3.png")
    result_path          = os.path.join(base_dir, "1-4.png")

    try:
        # Segmentasyon tahmini yapılacaksa
        process_and_save_image(original_image_path, mask_input_path, model)
    except Exception as e:
        print(f"[{idx}] Tahmin hatası: {e}")
        continue

    try:
        # Temizleme işlemi
        final_mask = process_mask(
            mask_path=mask_input_path,
            out_path=mask_processed_path,
            kernel_size=5,
            apply_closing=True,
            process_colors=[(255, 0, 0), (0, 255, 0), (0, 0, 255)]
        )
    except Exception as e:
        print(f"[{idx}] Maske temizleme hatası: {e}")
        continue

    try:
        mask_out_red_blue(original_image_path, mask_processed_path, out_path=result_path)
    except Exception as e:
        print(f"[{idx}] Maskeleme hatası: {e}")
        continue

    if show_images:
        try:
            original_img = cv2.cvtColor(cv2.imread(original_image_path), cv2.COLOR_BGR2RGB)
            processed_mask_img = cv2.cvtColor(cv2.imread(mask_processed_path), cv2.COLOR_BGR2RGB)
            result_img = cv2.cvtColor(cv2.imread(result_path), cv2.COLOR_BGR2RGB)

            fig, axs = plt.subplots(1, 3, figsize=(18, 6))
            axs[0].imshow(original_img)
            axs[0].set_title("Orijinal 1-1.png")
            axs[0].axis('off')

            axs[1].imshow(processed_mask_img)
            axs[1].set_title("Processed Mask 1-3.png")
            axs[1].axis('off')

            axs[2].imshow(result_img)
            axs[2].set_title("Masked Result 1-4.png")
            axs[2].axis('off')

            plt.tight_layout()
            plt.show()
        except Exception as e:
            print(f"[{idx}] Görüntü gösterme hatası: {e}")

İşlem ilerlemesi: 100%|██████████| 284/284 [02:40<00:00,  1.77resim/s]


In [9]:
import pandas as pd 
########## CALC TRAIN ##########
# CSV dosyasını oku
merged_df_calcTrain = pd.read_csv(r'N:\CBISDDSM\calcTrain\calc_train_updated.csv')

# İlk satırdaki tam dosya yolunu alalım
full_path = merged_df_calcTrain.loc[0, "File Location_y"]
print("Full path:", full_path)

# Sadece dizin kısmını almak için:
base_dir = os.path.dirname(full_path)
print("Base directory:", base_dir)

Full path: N:\CBISDDSM\calcTrain\CBIS-DDSM\Calc-Training_P_00005_RIGHT_CC\08-07-2016-DDSM-NA-23157\1.000000-full mammogram images-38548\1-1.dcm
Base directory: N:\CBISDDSM\calcTrain\CBIS-DDSM\Calc-Training_P_00005_RIGHT_CC\08-07-2016-DDSM-NA-23157\1.000000-full mammogram images-38548


In [10]:
# Görsel gösterimi aç/kapat
show_images = False

# İlk 15 örnek için işlem
#for idx in tqdm(range(15), desc="İşlem ilerlemesi", unit="resim",leave=True):
for idx in tqdm(range(len(merged_df_calcTrain)), desc="İşlem ilerlemesi", unit="resim", leave=True):
    full_path = merged_df_calcTrain.loc[idx, "File Location_y"]
    base_dir = os.path.dirname(full_path)
    
    original_image_path = os.path.join(base_dir, "1-1.png")
    mask_input_path      = os.path.join(base_dir, "1-2.png")
    mask_processed_path  = os.path.join(base_dir, "1-3.png")
    result_path          = os.path.join(base_dir, "1-4.png")

    try:
        # Segmentasyon tahmini yapılacaksa
        process_and_save_image(original_image_path, mask_input_path, model)
    except Exception as e:
        print(f"[{idx}] Tahmin hatası: {e}")
        continue

    try:
        # Temizleme işlemi
        final_mask = process_mask(
            mask_path=mask_input_path,
            out_path=mask_processed_path,
            kernel_size=5,
            apply_closing=True,
            process_colors=[(255, 0, 0), (0, 255, 0), (0, 0, 255)]
        )
    except Exception as e:
        print(f"[{idx}] Maske temizleme hatası: {e}")
        continue

    try:
        mask_out_red_blue(original_image_path, mask_processed_path, out_path=result_path)
    except Exception as e:
        print(f"[{idx}] Maskeleme hatası: {e}")
        continue

    if show_images:
        try:
            original_img = cv2.cvtColor(cv2.imread(original_image_path), cv2.COLOR_BGR2RGB)
            processed_mask_img = cv2.cvtColor(cv2.imread(mask_processed_path), cv2.COLOR_BGR2RGB)
            result_img = cv2.cvtColor(cv2.imread(result_path), cv2.COLOR_BGR2RGB)

            fig, axs = plt.subplots(1, 3, figsize=(18, 6))
            axs[0].imshow(original_img)
            axs[0].set_title("Orijinal 1-1.png")
            axs[0].axis('off')

            axs[1].imshow(processed_mask_img)
            axs[1].set_title("Processed Mask 1-3.png")
            axs[1].axis('off')

            axs[2].imshow(result_img)
            axs[2].set_title("Masked Result 1-4.png")
            axs[2].axis('off')

            plt.tight_layout()
            plt.show()
        except Exception as e:
            print(f"[{idx}] Görüntü gösterme hatası: {e}")

İşlem ilerlemesi: 100%|██████████| 1227/1227 [12:09<00:00,  1.68resim/s]
